Перед вами датасет, описывающий поступление детей в колледж.


Целевая переменная in_college.

Осмотрите данные.

Примените для предобработки пайплайн с прошлого урока.

Решите задачу классификации методами:
- логистической регрессии
- линейной классификации
- деревом решений
- методом опорных векторов

Сравните полученные в каждом решении метрики между собой

Попробуйте использовать GridSearch для поиска оптимальных гиперпараметров, перечисленных выше алгоритмов.


In [ ]:
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score, precision_recall_curve, roc_curve, roc_auc_score, \
              auc, mean_squared_error

import os
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
dtf = pd.read_csv('data.csv')

In [ ]:
dtf.head()

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college
0,Academic,A,Male,Less Interested,Urban,56,6950000,83.0,84.09,False,True
1,Academic,A,Male,Less Interested,Urban,57,4410000,76.8,86.91,False,True
2,Academic,B,Female,Very Interested,Urban,50,6500000,80.6,87.43,False,True
3,Vocational,B,Male,Very Interested,Rural,49,6600000,78.2,82.12,True,True
4,Academic,A,Female,Very Interested,Urban,57,5250000,75.1,86.79,False,False


In [ ]:
dtf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   bool   
 10  in_college             1000 non-null   bool   
dtypes: bool(2), float64(2), int64(2), object(5)
memory usage: 72.4+ KB


In [ ]:
dict = {True: 1, False: 0}

In [ ]:
dtf['parent_was_in_college'] = dtf['parent_was_in_college'].map(dict)

In [ ]:
dtf['in_college'] = dtf['in_college'].map(dict)

In [ ]:
dtf.head()

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college,in_college
0,Academic,A,Male,Less Interested,Urban,56,6950000,83.0,84.09,0,1
1,Academic,A,Male,Less Interested,Urban,57,4410000,76.8,86.91,0,1
2,Academic,B,Female,Very Interested,Urban,50,6500000,80.6,87.43,0,1
3,Vocational,B,Male,Very Interested,Rural,49,6600000,78.2,82.12,1,1
4,Academic,A,Female,Very Interested,Urban,57,5250000,75.1,86.79,0,0


In [ ]:
dtf['type_school'].value_counts()

Academic      609
Vocational    391
Name: type_school, dtype: int64

In [ ]:
dtf['school_accreditation'].value_counts()

B    519
A    481
Name: school_accreditation, dtype: int64

In [ ]:
dtf['interest'].value_counts()

Very Interested     324
Uncertain           261
Less Interested     229
Quiet Interested    100
Not Interested       86
Name: interest, dtype: int64

In [ ]:
x = dtf.drop(['in_college'], axis = 1)

In [ ]:
y = dtf['in_college']

In [ ]:
x

,type_school,school_accreditation,gender,interest,residence,parent_age,parent_salary,house_area,average_grades,parent_was_in_college
0,Academic,A,Male,Less Interested,Urban,56,6950000,83.0,84.09,0
1,Academic,A,Male,Less Interested,Urban,57,4410000,76.8,86.91,0
2,Academic,B,Female,Very Interested,Urban,50,6500000,80.6,87.43,0
3,Vocational,B,Male,Very Interested,Rural,49,6600000,78.2,82.12,1
4,Academic,A,Female,Very Interested,Urban,57,5250000,75.1,86.79,0
...,...,...,...,...,...,...,...,...,...,...
995,Vocational,A,Female,Very Interested,Rural,49,7420000,63.6,85.99,1
996,Academic,B,Female,Less Interested,Rural,51,7480000,84.3,89.72,1
997,Vocational,A,Male,Less Interested,Urban,49,5550000,75.2,79.56,0
998,Academic,B,Male,Uncertain,Rural,53,5840000,105.8,87.18,1


In [ ]:
y

0      1
1      1
2      1
3      1
4      0
      ..
995    1
996    1
997    1
998    1
999    0
Name: in_college, Length: 1000, dtype: int64

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 78.2+ KB


In [ ]:
numeric_features = x.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x.select_dtypes(include=['object']).columns

In [ ]:
numeric_features

Index(['parent_age', 'parent_salary', 'house_area', 'average_grades',
       'parent_was_in_college'],
      dtype='object')

In [ ]:
categorical_features

Index(['type_school', 'school_accreditation', 'gender', 'interest',
       'residence'],
      dtype='object')

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(x, y, random_state=0)

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
lr = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', LinearRegression())])

In [ ]:
lr.fit(X_train, y_train)
print("model score: %.3f" % lr.score(X_test, y_test))

model score: 0.506


# Классификация

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score, cross_val_predict

**Логистическая**

In [ ]:
logreg = LogisticRegression(random_state = 50)
logreg = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', logreg)])

logreg.fit(X_train, y_train)
logreg.score(X_test, y_test)

logreg_score_roc = cross_val_score(logreg, X_train, y_train, cv=5, scoring='roc_auc') #scoring='roc_auc'
logreg_score_f1 = cross_val_score(logreg, X_train, y_train, cv=5, scoring='f1') #scoring='f1'

In [ ]:
print(logreg_score_roc.mean())
print(logreg_score_f1.mean())

0.9393722648964754
0.8623854746820389


**Линейная**

In [ ]:
sgd = SGDClassifier(random_state = 52)

sgd = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', sgd)])

sgd.fit(X_train, y_train)
sgd.score(X_test, y_test)

sgd_score_roc = cross_val_score(sgd, X_train, y_train, cv=5, scoring='roc_auc') #scoring='roc_auc'
sgd_score_f1 = cross_val_score(sgd, X_train, y_train, cv=5, scoring='f1') #scoring='f1'

In [ ]:
print(sgd_score_roc.mean())
print(sgd_score_f1.mean())

0.9111383025130394
0.8222697182175069


**Дерево**

In [ ]:
dtclf = DecisionTreeClassifier(random_state = 44)

dtclf = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', dtclf)])

dtclf.fit(X_train, y_train)
dtclf.score(X_test, y_test)

dtclf_score_roc = cross_val_score(dtclf, X_train, y_train, cv=5, scoring='roc_auc') #scoring='roc_auc'
dtclf_score_f1 = cross_val_score(dtclf, X_train, y_train, cv=5, scoring='f1') #scoring='f1'

In [ ]:
print(dtclf_score_roc.mean())
print(dtclf_score_f1.mean())

0.8425267899478426
0.839238788475936


**Опорные вектора**

In [ ]:
svc = SVC(random_state = 43)

svc = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', svc)])

svc.fit(X_train, y_train)
svc.score(X_test, y_test)

svc_score_roc = cross_val_score(svc, X_train, y_train, cv=5, scoring='roc_auc') #scoring='roc_auc'
svc_score_f1 = cross_val_score(svc, X_train, y_train, cv=5, scoring='f1') #scoring='f1'

In [ ]:
print(svc_score_roc.mean())
print(svc_score_f1.mean())

0.9708417196143511
0.9063475215651737


# GreadSwarch и гиперпараметры

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
xx = preprocessor.fit_transform(x)

**Логистическая**

In [ ]:
log_params = {'max_iter': range(10, 15), 'n_jobs': range(-1, 5), 'penalty': ["l1", "l2", "elasticnet", "none"], 'multi_class': ["ovr", "multinomial"]}

In [ ]:
log = LogisticRegression()

In [ ]:
log_grid = GridSearchCV(log, log_params, cv=5, verbose=True, scoring='roc_auc')

In [ ]:
log_grid.fit(xx, y); # выдаёт огромное количество предупреждений

In [ ]:
log_grid.best_params_

{'max_iter': 11, 'multi_class': 'multinomial', 'n_jobs': -1, 'penalty': 'l2'}

**Линейная**

In [ ]:
sg_params = {'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'], 'penalty': ["l1", "l2", "elasticnet", "none"],
             'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]}

In [ ]:
sg = SGDClassifier()

In [ ]:
sg_grid = GridSearchCV(sg, sg_params, cv = 5, verbose=True, scoring='roc_auc')

In [ ]:
sg_grid.fit(xx, y)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/l

GridSearchCV(cv=5, estimator=SGDClassifier(),
             param_grid={'l1_ratio': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                                      0.9, 1],
                         'loss': ['hinge', 'log', 'modified_huber',
                                  'squared_hinge', 'perceptron', 'squared_loss',
                                  'huber', 'epsilon_insensitive',
                                  'squared_epsilon_insensitive'],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none']},
             scoring='roc_auc', verbose=True)

In [ ]:
sg_grid.best_params_

{'l1_ratio': 0.3, 'loss': 'log', 'penalty': 'l2'}

**Дерево**

In [ ]:
tree_params = {'max_depth': range(1,11), 'max_features': range(3,19)}

In [ ]:
tree = DecisionTreeClassifier()

In [ ]:
tree_grid = GridSearchCV(tree, tree_params, cv=5)# scoring='roc_auc'

In [ ]:
tree_grid.fit(xx, y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 11),
                         'max_features': range(3, 19)},
             scoring='roc_auc', verbose=True)

In [ ]:
tree_grid.best_params_

{'max_depth': 5, 'max_features': 12}

**Опорные вектора**

In [ ]:
svm_params = {'C': range(10,50), 'max_iter': range(3,19)}

In [ ]:
svm = SVC()

In [ ]:
svm_grid = GridSearchCV(svm, svm_params, cv=5)# scoring='roc_auc'

In [ ]:
svm_grid.fit(xx, y)

Выходные данные были обрезаны до нескольких последних строк (5000).
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=15).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=15).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=15).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=15).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarn

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': range(10, 50), 'max_iter': range(3, 19)})

In [ ]:
svm_grid.best_params_

{'C': 11, 'max_iter': 12}